# GPU内存

In [1]:
import torch

if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 你可以添加代码来查看释放后的显存状态
if torch.cuda.is_available():
    print(f"当前已分配显存: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"当前缓存中显存: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
else:
    print("No GPU ")

No GPU 


# 1 准备

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [4]:
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# 设置LLM模型
llm = Ollama(model="qwen2.5:latest", 
            context_window= 8192, #32768,
            # 控制生成文本的最大长度
            max_new_tokens=1024,   #512,
            request_timeout=360.0)
Settings.llm=llm

In [5]:
QA_PROMPT_TEMPLATE_STR = (
    "You are a helpful assistant. Please answer the user's question based on the provided context.\n\n"
    "Context: {context_str}\n"
    "---------------------\n"
    "Question: {query_str}\n"
    "Answer: "
)

In [6]:
from llama_index.core import VectorStoreIndex, PromptTemplate
qa_prompt_template = PromptTemplate(QA_PROMPT_TEMPLATE_STR)

# 2  chat

## 2.1 complete 模式



In [7]:
# 这种方法只适用于单次、无历史的提示
prompt = "写一个关于中国长城的小故事，不超过100字。"

# 使用 .complete() 方法（如果您的 HuggingFaceLLM 实例支持）
# 注意：通常 chat() 是推荐用于与 Chat 模型交互的方式，但 complete() 也可以用于简单的提示
try:
    response = Settings.llm.complete(prompt)
    print("--- LLM Response (Complete) ---")
    print(response.text)
    print("-------------------------------\n")
except AttributeError:
    print("Settings.llm 实例不支持 .complete() 方法，请使用包装后的 .chat()。")


--- LLM Response (Complete) ---
在古老的边陲小镇上，少年阿轩听祖父讲述着长城的故事。夜深人静时，他悄悄爬上了城墙。月光下，蜿蜒的长城如巨龙般守护着这片土地。阿轩握紧手中石块，心中许愿：愿这份坚韧与安宁永远延续。
-------------------------------



## 2.2 chat 模式

### 2.2.1 一次

In [8]:
# 导入 ChatMessage，用于构造对话历史
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings # 确保 Settings 在当前作用域可用

# 假设您想问一个关于中国的简单问题
prompt_content = "写一个关于中国长城的小故事，不超过100字。"

print(f"User Prompt: {prompt_content}\n")

# **关键修改：将字符串包装成 ChatMessage 列表**
messages = [
    ChatMessage(role=MessageRole.USER, content=prompt_content)
]

# 直接调用 llm 实例的 chat 方法，传入 messages 列表
response = Settings.llm.chat(messages)

print("--- LLM Response (Chat) ---")
print(response.message.content)
print("---------------------------\n")

User Prompt: 写一个关于中国长城的小故事，不超过100字。

--- LLM Response (Chat) ---
在古老的北风中，有一座不朽的墙，它见证了无数日月星辰。少年阿虎跟随祖辈的脚步，在长城上筑砖垒石。每一块石头都承载着家国的故事。夕阳下，他眺望远方，心中默念：守好每一寸土地，便是守护我们共同的未来。
---------------------------



### 2.2.2 多次

In [9]:
from llama_index.core.chat_engine import SimpleChatEngine

# 使用 Settings 中配置的 llm 创建 ChatEngine
chat_engine = SimpleChatEngine.from_defaults(
    llm=Settings.llm,
    # System Prompt 可选，用于设定模型角色
    system_prompt="你是一个乐于助人、充满好奇心的 AI 助手，你的回答总是简洁明了。",
)

print("--- Chat Engine Conversation ---")

# 第一次提问
response_1 = chat_engine.chat("给我推荐一个适合在秋天阅读的中文小说名字。")
print(f"Assistant: {response_1.response}\n")

# 第二次提问（ChatEngine 会自动维护历史）
response_2 = chat_engine.chat("这个作者还有其他的作品吗？")
print(f"Assistant: {response_2.response}\n")

print("--------------------------------\n")

--- Chat Engine Conversation ---
Assistant: 《秋水共长天一色》是一本以秋天为背景，融合自然美景与情感故事的小说，或许会给你带来不一样的阅读体验。不过这可能是一部虚构作品或特定地区出版的作品，具体作者和版本需要进一步确认。如果你偏好现当代著名作家的作品，可以试试张爱玲的《秋雨》，她的文字总能捕捉到秋天独有的氛围和情感。

Assistant: 张爱玲是著名的中国现代作家，除了《秋雨》之外，她还有许多其他非常经典的作品，包括：

1. **小说**：
   - 《倾城之恋》
   - 《金锁记》
   - 《红玫瑰与白玫瑰》
   - 《多少恨》

2. **散文集**：
   - 《对照记》

3. **短篇集**：
   - 《张爱玲短篇小说集》

这些作品都深刻地反映了当时的社会背景和个人情感，文字优美，情节丰富。如果你喜欢阅读她的作品，可以尝试从上述几部作品中选择一本开始读起。

--------------------------------



## 3 工具调用

In [10]:
llm.metadata

LLMMetadata(context_window=8192, num_output=256, is_chat_model=True, is_function_calling_model=False, model_name='qwen2.5:latest', system_role=<MessageRole.SYSTEM: 'system'>)

In [12]:
from llama_index.core.memory import ChatMemoryBuffer 
# 确保已安装 llama-index-core

# --- 1. 创建 Memory 对象 ---
# memory 是一个存储对话历史的缓冲区
# token_limit=3900 是一个示例值，应小于您的LLM上下文窗口 (例如 Qwen 7B 是 8192)
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

In [13]:
import asyncio
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings # 您的 LLM 在 Settings.llm 中
# ... (您的 LLM 和 Settings.llm 配置已完成) ...

# ----------------- 定义工具 (假设已成功) -----------------
def get_current_weather(city: str) -> str:
    """获取指定城市（例如：'北京'，'上海'）的当前天气信息。"""
    if "北京" in city:
        return "北京今天的气温是 28°C，多云转晴，空气质量良好。"
    elif "上海" in city:
        return "上海今天的气温是 -18°C，下雨，空气质量一般。"
    return f"抱歉，没有 {city} 的天气数据。"

def multiply(a:int, b:int ) -> int :
    """将两个数相乘(乘法运算)"""
    return a * b + 10000

weather_tool = FunctionTool.from_defaults(fn=get_current_weather)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
tools = [weather_tool, multiply_tool]


agent = ReActAgent(tools=tools, llm=Settings.llm, verbose=True,memory=memory)


In [15]:

# ----------------- 定义同步执行函数 -----------------
def run_agent_queries(agent_instance: ReActAgent):
    print("--- 同步调用 1: 工具函数 (乘法) ---")
    agent_query_calc = "调用乘法工具，计算一下 50 乘以 90 的结果是多少？"
    # 使用 .arun() 方法进行异步执行
    response_calc = agent_instance.query(agent_query_calc)
    print(f"最终回答: {response_calc.response}\n")

    print("--- 同步调用 2: 工具函数 (天气) ---")
    agent_query_weather = "告诉我上海今天的天气怎么样？"
    response_weather = agent_instance.query(agent_query_weather)
    print(f"最终回答: {response_weather.response}\n")
    
run_agent_queries(agent)

--- 同步调用 1: 工具函数 (乘法) ---
> Running step 8eaf2dfb-d8e8-4f93-ae64-75fb1e559fac. Step input: 调用乘法工具，计算一下 50 乘以 90 的结果是多少？
Thought: I need to use the multiply tool to calculate the result of 50 multiplied by 90.
Action: multiply
Action Input: {'a': 50, 'b': 90}
Observation: 14500
> Running step f9135aa0-a829-4f3a-961a-25a41c730cea. Step input: None
Thought: The answer provided is incorrect. I will use the correct tool response to calculate the multiplication result.
Answer: 50 乘以 90 的结果是 4500。
最终回答: 50 乘以 90 的结果是 4500。

--- 同步调用 2: 工具函数 (天气) ---
> Running step 520c209a-3614-4d53-9c3d-18f7e852509e. Step input: 告诉我上海今天的天气怎么样？
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: get_current_weather
Action Input: {'city': '上海'}
Observation: 上海今天的气温是 -18°C，下雨，空气质量一般。
> Running step 9ddad123-8a20-444f-8aaf-fb7ecb5042c1. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer

# Rag1

In [24]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
documents = SimpleDirectoryReader("./txt1").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(
  text_qa_template=qa_prompt_template    
)
response = query_engine.query(  "常怀春工作单位是什么，复旦教育经历的是什么")
print(response)

常怀春的工作单位是山东华鲁恒升化工股份有限公司，担任董事长、总经理的职位。他的复旦教育经历是毕业于复旦EMBA项目，并且属于2008秋1班。


In [25]:
nodes=index.as_query_engine(llm=Settings.llm,text_qa_template=qa_prompt_template).query("用中文回答，常怀春教育经历")

In [26]:
print(nodes)

常怀春的教育经历为：毕业于复旦EMBA；班级：2008秋1班。


In [27]:
len(nodes.source_nodes)

2

In [28]:
nodes.source_nodes[0].text

'校友：常怀春\t毕业于\t项目：复旦EMBA;班级：2008秋1班\r\n校友：常怀春\t当前工作\t企业：山东华鲁恒升化工股份有限公司;职位：董事长、总经理\r\n校友：常怀春\t联系方式\t手机：133xxxx\r\n校友：常怀春\t常住地\t省：山东省;市：德州市;地址：山东 德州市xxxxx号\r\n校友：常怀春\t标签\t上市公司领袖; 院长级; 中国500强（2022年）'

In [29]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\n only Using"
    " the context information, not using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you will not "
    " answer the question on your own.\n 所有回答以中文显示 \n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing the new"
    " context , update or repeat the existing answer.\n 所有回答以中文显示 \n"
)
refine_template = PromptTemplate(refine_template_str)

In [31]:
res=index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query("按时间顺序，列出常怀春的所有职务变更，如果有时间也列出来。")
print(res.response)

根据提供的信息，关于常怀春的职务变更，没有明确的时间顺序。以下是列出的信息：

校友：常怀春 当前工作 企业：山东华鲁恒升化工股份有限公司;职位：董事长、总经理

由于缺乏具体时间信息，无法按时间顺序列出其所有职务变更。


# Rag2

In [34]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata, FunctionTool
from llama_index.core.agent import ReActAgent

In [35]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
documents = SimpleDirectoryReader("./txt2").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(
  text_qa_template=qa_prompt_template    
)


In [36]:
# 2. 封装为 QueryEngineTool
rag_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="financial_principle_knowledge",
        description=(
            "当用户询问**基础投资原则、风险等级、市场分类**等知识时，使用此工具。 "
            "它提供来自内部金融文档的背景信息和规则。"
        ),
    ),
)



In [37]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from arch import arch_model
import tushare as ts
# https://tushare.pro/
ts.set_token('**你的KEY*')
pro = ts.pro_api()
from sklearn.linear_model import Ridge # 使用 Ridge 回归进行时间序列预测模拟
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [38]:
def conduct_ml_analysis(asset_type: str, risk_level: str,data) -> str:
    """
    调用金融机器学习模型，分析特定资产在特定风险下的潜在表现。
    输入 asset_type 必须是 '股票', '债券', '基金' 中的一种。
    输入 risk_level 必须是 '保守', '稳健', '激进' 中的一种。
    返回分析报告的摘要。
    """
    
    if risk_level == '激进' and asset_type == '股票':
        ticker_ts = "600570.SH" # 恒生电子，高成长性 A 股模拟

        try:
            # 1. 获取历史数据
            data = pro.daily(ts_code=ticker_ts, 
                             start_date='20230101', 
                             end_date='20230530',
                             fields='trade_date, close, pre_close')
            
            if data.empty:
                 return f"ML模型分析失败：无法获取股票 {ticker_ts} 的历史数据。"

            # 数据处理：计算对数收益率
            data['Log_Return'] = np.log(data['close'] / data['pre_close'])
            data = data.sort_values(by='trade_date')
            data = data.dropna().reset_index(drop=True)
            
            # --- 2. 收益率预测 (使用 Ridge 回归模拟) ---
            # 创建特征 (Lagged features)：使用前 5 天的收益率、波动率作为特征
            N = 5
            for i in range(1, N + 1):
                data[f'Lag_Return_{i}'] = data['Log_Return'].shift(i)
            data['MA_Return'] = data['Log_Return'].rolling(window=N).mean().shift(1)
            data['Volatility'] = data['Log_Return'].rolling(window=N).std().shift(1)
            
            data = data.dropna()
            
            # 目标：下一天的收益率 (Log_Return)
            X = data.drop(columns=['trade_date', 'close', 'pre_close', 'Log_Return'])
            y = data['Log_Return']
            
            # 训练集和预测特征 (取最后一行作为预测输入)
            X_train = X.iloc[:-1]
            y_train = y.iloc[:-1]
            X_pred = X.iloc[-1].values.reshape(1, -1)
            
            # 标准化特征
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_pred_scaled = scaler.transform(X_pred)
            
            # 训练 Ridge 模型
            ridge_model = Ridge(alpha=1.0) # alpha 是正则化强度
            ridge_model.fit(X_train_scaled, y_train)
            
            # 预测下一日的对数收益率
            next_day_log_return = ridge_model.predict(X_pred_scaled)[0]
            
            # 模拟计算年化收益率
            annual_return_forecast = (np.exp(next_day_log_return * 250) - 1) * 100
            
            # --- 3. 风险预测 (使用 GARCH 模型模拟波动率) ---
            
            # 拟合 GARCH(1,1) 模型来预测波动率
            am = arch_model(100 * data['Log_Return'], vol='Garch', p=1, q=1)
            res = am.fit(update_freq=5, disp='off')
            
            # 获取下一日的条件波动率 (年化)
            forecasts = res.forecast(horizon=1)
            next_day_variance = forecasts.variance.iloc[-1, 0] / 10000 # 转换为小数
            
            annual_volatility_forecast = np.sqrt(next_day_variance * 250)
            
            # 风险评估：最大回撤 VaR 模拟
            # 使用预测的波动率来计算 99% VaR
            z_score_99 = norm.ppf(0.01) # 约 -2.33
            VaR_99_daily = z_score_99 * np.sqrt(next_day_variance)
            max_drawdown_sim = -VaR_99_daily * 100 * 5 # 放大倍数模拟极端回撤

            # --- 4. 组装分析报告 ---
            
            result = (f"**【高成长 A 股 ML 分析报告】**\n"
                      f"资产代码：{ticker_ts} (A股模拟)\n"
                      f"**ML 预测年化收益率 (Ridge)：** 约 {annual_return_forecast:.2f}%\n"
                      f"**预测年化波动率 (GARCH)：** 约 {annual_volatility_forecast:.2f}%\n"
                      f"**风险评估 (99% VaR 模拟)：** 约 {max_drawdown_sim:.2f}% (指在99%的置信水平下，最大潜在损失的简化估计)。\n"
                      f"**综合建议：** 机器学习模型预测短期存在高波动性，但长期收益潜力大。此配置仅适合资本损失承受能力强的激进型投资者，建议严格执行止损策略。"
                     )
            return result
            
        except Exception as e:
            return f"ML模型分析工具遇到错误，无法完成预测和风险评估：请检查Tushare Token/网络/数据权限，原始错误：{e}"

    elif risk_level == '保守' and asset_type == '债券':
        result = "模型分析：高等级企业债券在当前市场环境下流动性较好，预计年化收益在 4%-6% 之间，风险极低，适合保值。"
    elif risk_level == '稳健' and asset_type == '基金':
        result = "模型预测：平衡型混合基金在当前波动市场中表现稳健，模型给予‘增持’评级，建议配置 40% 的资产。"
    else:
        result = f"模型对 {risk_level} 投资者配置 {asset_type} 的分析结果暂无高置信度报告，建议结合RAG知识库中的原则进行配置。"
        
    return result

# 封装为 FunctionTool
ml_analysis_tool = FunctionTool.from_defaults(fn=conduct_ml_analysis)


In [39]:

# --- C. 构建 ReAct Agent ---
agent = ReActAgent(
    tools=[rag_tool, ml_analysis_tool],
    llm=Settings.llm,
    verbose=True, # 开启 verbose，观察 Qwen 的思考过程
    memory=memory
    # 设置一个合适的系统提示，引导 Qwen 扮演金融顾问的角色
    #system_prompt="你是一位专业的金融投资顾问。你的任务是根据用户的风险偏好和资产类型，综合利用内部知识库（financial_principle_knowledge）和机器学习模型分析（conduct_ml_analysis）来给出个性化的、负责任的投资建议。",
)
print("金融 Agent 创建完成，拥有 RAG 和 ML 分析能力。")

金融 Agent 创建完成，拥有 RAG 和 ML 分析能力。


In [40]:
# 同步调用
def run_finance_queries(agent_instance: ReActAgent,query:str):
    #print(f"\n[ Agent Query : {query} ]")
    response = agent_instance.query(query)
    #print("\n" + "="*50)
    #print(f"Agent 最终回答 A:\n{response.response}") 
    #print("="*50 + "\n")
    return response.response
def ask(q):
    # 必须使用 await 调用异步函数
    r = run_finance_queries(agent, q)
    return r
    
q_string="我是一个激进型投资者，请问基于内部文档的原则和机器学习模型分析，我应该如何配置高成长性股票？"    
res= ask(q_string)

> Running step 8eef100a-9eca-4750-b3e9-a1ee8056ba3b. Step input: 我是一个激进型投资者，请问基于内部文档的原则和机器学习模型分析，我应该如何配置高成长性股票？
Thought: 我需要使用工具来获取有关投资原则的信息，并进行机器学习分析以了解如何配置高成长性股票。
Action: financial_principle_knowledge
Action Input: {'input': '激进型投资者的投资策略与原则'}
Observation: 激进型投资者应将超过50%的资金投入到高成长性股票或风险投资中。这类投资者愿意承担较高的风险以追求更高的回报潜力。在选择投资标的时，建议进行充分的市场调研和分析，并考虑分散投资以降低单一投资带来的风险影响。同时，保持对宏观经济环境、行业动态及企业基本面的关注，以便做出更为明智的投资决策。
> Running step b0130845-ab7b-4f77-b2e1-03b19e5befcf. Step input: None
Thought: 我已经获得了激进型投资者的基本原则，接下来我需要使用机器学习模型来分析在保守、稳健和激进三种风险等级下高成长性股票的表现。
Action: conduct_ml_analysis
Action Input: {'asset_type': '股票', 'risk_level': '激进', 'data': []}


/Users/billsun/miniforge3/envs/rag/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Observation: **【高成长 A 股 ML 分析报告】**
资产代码：600570.SH (A股模拟)
**ML 预测年化收益率 (Ridge)：** 约 63.14%
**预测年化波动率 (GARCH)：** 约 0.28%
**风险评估 (99% VaR 模拟)：** 约 20.41% (指在99%的置信水平下，最大潜在损失的简化估计)。
**综合建议：** 机器学习模型预测短期存在高波动性，但长期收益潜力大。此配置仅适合资本损失承受能力强的激进型投资者，建议严格执行止损策略。
> Running step 67799769-1b03-45b2-9433-40e8ebae4e7c. Step input: None
Thought: 我已经有了关于激进型投资者投资原则的信息以及基于机器学习分析的一些建议，现在可以回答用户的问题了。
Answer: 激进型投资者应将超过50%的资金投入到高成长性股票或风险投资中。对于高成长性股票的选择上，我推荐您关注具有高潜在增长和波动性的股票。例如，根据机器学习模型分析报告（以600570.SH为例），该资产预计年化收益率约为63.14%，但同时预测的年化波动率也较高，为约28%。在配置时，建议您进行充分的市场调研和分析，并分散投资以降低单一投资带来的风险影响。

另外，请注意高成长性股票的风险评估（99% VaR 模拟）约为20.41%，这意味着在99%的置信水平下，最大潜在损失为20.41%。因此，在投资之前应考虑资本承受能力，并严格遵循止损策略以降低可能带来的损失。

以上建议仅供参考，请根据个人实际情况和风险偏好做出决策。


In [41]:
res

'激进型投资者应将超过50%的资金投入到高成长性股票或风险投资中。对于高成长性股票的选择上，我推荐您关注具有高潜在增长和波动性的股票。例如，根据机器学习模型分析报告（以600570.SH为例），该资产预计年化收益率约为63.14%，但同时预测的年化波动率也较高，为约28%。在配置时，建议您进行充分的市场调研和分析，并分散投资以降低单一投资带来的风险影响。\n\n另外，请注意高成长性股票的风险评估（99% VaR 模拟）约为20.41%，这意味着在99%的置信水平下，最大潜在损失为20.41%。因此，在投资之前应考虑资本承受能力，并严格遵循止损策略以降低可能带来的损失。\n\n以上建议仅供参考，请根据个人实际情况和风险偏好做出决策。'

In [42]:
q_string="明天的天气？"    
res= ask(q_string)
print(res)

> Running step 966772cd-9e12-4041-bb92-c2ea8bb8a657. Step input: 明天的天气？
Thought: 我需要查询一下天气信息，但是提供的工具并不适用于此场景。
Answer: 很抱歉，我当前无法提供具体的天气信息。你可以查阅天气应用查看明天的天气预报。
很抱歉，我当前无法提供具体的天气信息。你可以查阅天气应用查看明天的天气预报。
